In [1]:
import pandas as pd
import json
import pickle
import numpy as np
import requests
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import glob
import collections
from tqdm import tqdm
import re

In [2]:
folder_url = "../../../data/OpenAI/Urls/"

caribbean_url_dict = pickle.load(open(F"{folder_url}urls_caribbean.pkl", 'rb'))
palms_url_dict = pickle.load(open(F"{folder_url}urls_palms.pkl", 'rb'))
plantnet_url_dict = pickle.load(open(F"{folder_url}urls_plantnet.pkl", 'rb'))

In [3]:
# Test

# caribbean_url_dict = {k: caribbean_url_dict[k][0:5] for k in list(caribbean_url_dict)[:4]}
# palms_url_dict = {k: palms_url_dict[k][0:5] for k in list(palms_url_dict)[:4]}
# plantnet_url_dict = {k: plantnet_url_dict[k][0:5] for k in list(plantnet_url_dict)[:4]}

In [4]:
def extract_paragraphs_from_urls(url_dict):

    # Init dict
    text_dict = collections.defaultdict(list)

    # Loop over species and URLS
    for (species, urls) in tqdm(url_dict.items(), desc="Species", leave=True, position=0):
        for url in tqdm(urls, desc="URL", leave=False, position=0):

            try:
                session = requests.Session()
                retry = Retry(total=1,
                              connect=1, 
                              backoff_factor=0.5)
                adapter = HTTPAdapter(max_retries=retry)
                session.mount('http://', adapter)
                session.mount('https://', adapter)
                response = session.get(url, timeout=3)
                # response = requests.get(url)
                
                soup = BeautifulSoup(response.text, 'html.parser')
                paragraphs = soup.find_all('p')
                for paragraph in paragraphs:
                    text_dict[species].append(paragraph.text)

            except:
                text_dict[species].append("Invalid URL")

    return text_dict

In [5]:
caribbean_text_dict = extract_paragraphs_from_urls(caribbean_url_dict)
# palms_text_dict = extract_paragraphs_from_urls(palms_url_dict)
# plantnet_text_dict = extract_paragraphs_from_urls(plantnet_url_dict)

URL:  70%|███████   | 28/40 [00:22<00:11,  1.03it/s]/it]/usr/local/anaconda3/envs/webscraping/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
Species: 100%|██████████| 42/42 [33:14<00:00, 47.50s/it]


In [6]:
folder_text = "../../../data/OpenAI/TextSnippets/"

with open(F"{folder_text}paragraphs_caribbean.pkl", 'wb') as f:
    pickle.dump(caribbean_text_dict, f)

# with open(F"{folder_text}paragraphs_palms.pkl", 'wb') as f:
#     pickle.dump(palms_text_dict, f)

# with open(F"{folder_text}paragraphs_plantnet.pkl", 'wb') as f:
#     pickle.dump(plantnet_text_dict, f)